In [1]:
# gestione missing values:
# togliere campione,
# se per 900 campioni abbiamo solo 10 campioni e ognuno di questi ha una sola domanda alla quale non ha risposto
#       si posso gestire in maniera tale da non perdere il campione,
#       oppure talmente pochi che si possono eliminare
# tirar fuori un altra colonna, rapporto tra risposta fra una colonna e un'altra
#       analizza l'efficenza su questo calcolo
# o raggruppamento per ruolo/professione in maniera efficiente e non
# statistica esplorativa


In [2]:
#non considerare Use5
#prevedere se prof è d'accordo con l'apprendimento con wiki dalle altre domande
# in input tutto cio che non è "use" - output: si seleziona qualcuno di questi use e si cerca di predirli

# tipo di docente ha rilevanza nella decisione

# Titolo

elenco:
- Import del Dataset e prime analisi
- Manipolazione e pulizia del dataset


## Import del Dataset e prime analisi

In [3]:
import numpy as np
import pandas as pd
from IPython.display import display
import main

data = pd.read_csv('dataset/uci - missing data/wiki4HE/wiki4HE.csv', sep=';')
print(data)

     AGE  GENDER DOMAIN  PhD YEARSEXP  UNIVERSITY UOC_POSITION OTHER_POSITION  \
0     40       0      2    1       14           1            2              ?   
1     42       0      5    1       18           1            2              ?   
2     37       0      4    1       13           1            3              ?   
3     40       0      4    0       13           1            3              ?   
4     51       0      6    0        8           1            3              ?   
..   ...     ...    ...  ...      ...         ...          ...            ...   
908   43       0      5    1       21           2            ?              ?   
909   53       0      6    0       25           2            ?              ?   
910   39       0      5    1        9           2            ?              ?   
911   40       0      3    1       10           2            ?              ?   
912   41       1      3    1       12           2            ?              ?   

    OTHERSTATUS USERWIKI  .

In [4]:
pd.set_option('display.max_columns', 54)
data.replace('?', np.nan, inplace=True)
print(data.head())


   AGE  GENDER DOMAIN  PhD YEARSEXP  UNIVERSITY UOC_POSITION OTHER_POSITION  \
0   40       0      2    1       14           1            2            NaN   
1   42       0      5    1       18           1            2            NaN   
2   37       0      4    1       13           1            3            NaN   
3   40       0      4    0       13           1            3            NaN   
4   51       0      6    0        8           1            3            NaN   

  OTHERSTATUS USERWIKI PU1 PU2 PU3 PEU1 PEU2 PEU3 ENJ1 ENJ2 Qu1 Qu2 Qu3 Qu4  \
0         NaN        0   4   4   3    5    5    3    4    4   3   3   2   2   
1         NaN        0   2   3   3    4    4    3    3    4   4   4   3   3   
2         NaN        0   2   2   2    4    4    3    3    3   2   2   2   5   
3         NaN        0   3   3   4    3    3    3    4    3   3   4   3   3   
4         NaN        1   4   3   5    5    4    3    4    4   4   5   4   3   

  Qu5 Vis1 Vis2 Vis3 Im1 Im2 Im3 SA1 SA2 SA3 Use1 

In [5]:
# vedere i tipi
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913 entries, 0 to 912
Data columns (total 53 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   AGE             913 non-null    int64 
 1   GENDER          913 non-null    int64 
 2   DOMAIN          911 non-null    object
 3   PhD             913 non-null    int64 
 4   YEARSEXP        890 non-null    object
 5   UNIVERSITY      913 non-null    int64 
 6   UOC_POSITION    800 non-null    object
 7   OTHER_POSITION  652 non-null    object
 8   OTHERSTATUS     373 non-null    object
 9   USERWIKI        909 non-null    object
 10  PU1             906 non-null    object
 11  PU2             902 non-null    object
 12  PU3             908 non-null    object
 13  PEU1            909 non-null    object
 14  PEU2            899 non-null    object
 15  PEU3            816 non-null    object
 16  ENJ1            906 non-null    object
 17  ENJ2            896 non-null    object
 18  Qu1       

In [6]:
#conto gli na
pd.DataFrame({ 'contain na ': data.isna().any() ,'number of na ': data.isna().sum() })

,contain na,number of na
AGE,False,0
GENDER,False,0
DOMAIN,True,2
PhD,False,0
YEARSEXP,True,23
UNIVERSITY,False,0
UOC_POSITION,True,113
OTHER_POSITION,True,261
OTHERSTATUS,True,540
USERWIKI,True,4


In [7]:
# numero di righe con almeno un na:
data.shape[0] - data.dropna().shape[0]


737

## Manipolazione e pulizia del dataset

In [8]:
# si potrebbe dividere anche in PROFESSORI E DOMANDE per poi riunirli con JOIN

domande = data.iloc[:, 10:]
prof = data.iloc[:, :10]

In [9]:

# University si indica l'uni di appartenenza
# se in UNI = 1 -> in UOC la posizione in UOC - probabilmente non hanno altri impieghi in altre uni
# se in UNI = 2 -> in UOC la posizione in UOC prob non esiste - hanno altri ruoli
#potrei percio creare una col con solo la posizione lavorativa


# tenere conto (sotto)

# maggior parte dei dati mancanti provengono dalla variabile "OTHER_POSITION".
# Questa variabile descrive il "lavoro come part-time in un'altra università e membri UPF".
# Questa è una domanda mal progettata in quanto non è applicabile
# alla maggior parte dei membri della facoltà e quindi non hanno risposto.
# Di conseguenza, rimuoveremo questa variabile dai dati per eliminare i dati mancanti sistematici.



### Pulizia degli attributi

In [11]:
# se non specificato considero gli anni di esperienza = 0
prof['YEARSEXP'] = prof['YEARSEXP'].fillna(0)

# elimina righe  con domain e userwiki na, 5 righe in meno (domain si potrebbe considerare come altro)
prof = prof[prof['DOMAIN'].notna() & prof['USERWIKI'].notna()]

#quanti campioni rimanenti
prof.shape

(907, 10)

### Corrispondenze sui nomi tabelle
Gli attributi "OTHER", "OTHER_POSITION" della descrizione non corrispondono a quelli presenti nel dataset:
 - __"OTHER"__ - non è presente;
 - __"OTHER_POSITION"__ - non rispecchia i valori attesi.

L'attributo "OTHER_POSITION" del dataset presenta solo due valori, ___1___ e ___2___, questi risultano essere associabili all' attributo "OTHER" nella descrizione.

In [12]:
prof['OTHER_POSITION'].dropna().astype(int).describe()

count    648.000000
mean       1.586420
std        0.492855
min        1.000000
25%        1.000000
50%        2.000000
75%        2.000000
max        2.000000
Name: OTHER_POSITION, dtype: float64

Nel dataset è presente un'altra colonna che non viene descritta sul sito, ovvero "OTHERSTATUS".
I dati di quest'ultima vanno da un minimo di ___1___ a un massimo di ___7___ e risultano essere uguali ai valori attesi dall'attibuto "OTHER_POSITION" del sito, ad esclusione del valore 7.

In [13]:
prof['OTHERSTATUS'].dropna().astype(int).describe()

count    372.000000
mean       4.201613
std        2.223302
min        1.000000
25%        2.000000
50%        4.000000
75%        7.000000
max        7.000000
Name: OTHERSTATUS, dtype: float64

#### Assunsioni
Assumiamo come veritiere le affermazioni precedenti e associamo i vari attributi alle colonne nel dataset:
- "OTHER" (descrizione) -> "OTHER_POSITION" (dataset)
- "OTHER_POSITION" (descrizione) -> "OTHERSTATUS" (dataset)

##

In [14]:
# quanti non è assunto da UOC?
prof[prof['UNIVERSITY'] == 2].shape

(112, 10)

In [15]:
# chi non è assunto in UOC -> data[data['UOC_POSITION'].isna()]
#tutti i professori che appartengono a UPF non hanno specificato un ruolo nella colonna "OTHER_POSITION"
pd.DataFrame({ 'contain na ': prof[prof['UNIVERSITY'] == 2].isna().any() ,'number of na ': prof[prof['UNIVERSITY'] == 2].isna().sum() })


,contain na,number of na
AGE,False,0
GENDER,False,0
DOMAIN,False,0
PhD,False,0
YEARSEXP,False,0
UNIVERSITY,False,0
UOC_POSITION,True,112
OTHER_POSITION,True,112
OTHERSTATUS,True,5
USERWIKI,False,0


In [16]:
prof['OTHERSTATUS'].dropna().astype(int)[prof['OTHERSTATUS'].dropna().astype(int)[:] == 7].count()


106

In [17]:

prof['UOC_POSITION'] = prof['UOC_POSITION'].fillna(0)
prof['OTHERSTATUS'] = prof['OTHERSTATUS'].fillna(0)
prof['OTHER_POSITION'] = prof['OTHER_POSITION'].fillna(0)

prof = prof.astype({'AGE': 'int','GENDER': 'int','DOMAIN': 'int','PhD': 'int','YEARSEXP': 'int','UNIVERSITY': 'int','UOC_POSITION': 'int','OTHER_POSITION': 'int','OTHERSTATUS': 'int','USERWIKI': 'int'})

prof.info()

# se si considera 7 come val buono
# prof[:][(prof["OTHERSTATUS"] == 0) & (prof["UOC_POSITION"] == 0)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 907 entries, 0 to 912
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   AGE             907 non-null    int32
 1   GENDER          907 non-null    int32
 2   DOMAIN          907 non-null    int32
 3   PhD             907 non-null    int32
 4   YEARSEXP        907 non-null    int32
 5   UNIVERSITY      907 non-null    int32
 6   UOC_POSITION    907 non-null    int32
 7   OTHER_POSITION  907 non-null    int32
 8   OTHERSTATUS     907 non-null    int32
 9   USERWIKI        907 non-null    int32
dtypes: int32(10)
memory usage: 42.5 KB


In [18]:
# prof["MAX POSITION"] = prof[["UOC_POSITION", "OTHERSTATUS"]].max(axis=1)
# prof[:][prof["MAX POSITION"] == 0]

In [19]:
# prof = prof.drop([813,828,830,835,882])
# prof = prof.drop(columns=['UOC_POSITION', 'OTHERSTATUS', 'OTHER_POSITION'])

In [20]:
# se non si considera 7 come val buon

prof['OTHERSTATUS'][ prof['OTHERSTATUS'] > 6] = 0 # considero il 7 come "altro" percio non lo considero

position = prof[["UOC_POSITION", "OTHERSTATUS"]].max(axis=1)
position[:].describe()

count    907.000000
mean       4.996692
std        1.793104
min        0.000000
25%        4.000000
50%        6.000000
75%        6.000000
max        6.000000
dtype: float64

In [21]:
# %timeit prof[["UOC_POSITION", "OTHERSTATUS"]].max(axis=1)

In [22]:
# %timeit main.merge_two_series_by_max(prof['OTHERSTATUS'].array, prof['UOC_POSITION'].array)


In [23]:
prof.insert(loc=6, column='POSITION', value=position)
prof = prof.drop(columns=['UOC_POSITION', 'OTHERSTATUS', 'OTHER_POSITION'])
prof = prof[prof['POSITION'] != 0] # professori che non hanno neanche un ruolo
prof.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 871 entries, 0 to 912
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   AGE         871 non-null    int32
 1   GENDER      871 non-null    int32
 2   DOMAIN      871 non-null    int32
 3   PhD         871 non-null    int32
 4   YEARSEXP    871 non-null    int32
 5   UNIVERSITY  871 non-null    int32
 6   POSITION    871 non-null    int32
 7   USERWIKI    871 non-null    int32
dtypes: int32(8)
memory usage: 34.0 KB


In [24]:
# mostra na di prof dopo manuipolazioni
pd.DataFrame({ 'contain na ': prof.isna().any() ,'number of na ': prof.isna().sum() })

,contain na,number of na
AGE,False,0
GENDER,False,0
DOMAIN,False,0
PhD,False,0
YEARSEXP,False,0
UNIVERSITY,False,0
POSITION,False,0
USERWIKI,False,0


In [25]:
prof.index


Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            898, 899, 900, 901, 905, 908, 909, 910, 911, 912],
           dtype='int64', length=871)

In [26]:
#Una semplice esclusione delle osservazioni (righe) con NA nelle risposte del sondaggio rimuove circa 300 osservazioni. Questo è circa 1/3 del set di dati ed è una parte abbastanza significativa dei dati.
# lo mostro soltanto per vedere quante sono le righe complete
data.dropna().shape

(176, 53)

In [27]:
# missing data su domande:
# - dropna (no - troppe eliminazioni)
# - eliminare riga solo se ha n risposte mancanti, con n valore arbitrario
# - media (se modello che vogliamo analizzare non ammette osservazioni mancanti - per alcune domande molti na - non mi convince)
# - metto tutti gli na a 0 o 6

In [28]:
#Esistono diversi modi per gestire i dati nulli nel set di dati della domanda.
# Un modo sarebbe trattare la scala likert come continua con valori compresi tra 1 e 5,
# quindi potremmo inserire la media di ogni domanda nei valori mancanti,
# come verrà fatto con i dati completi in seguito.
# Tuttavia, in questo modo non sarebbe possibile effettuare un'analisi sul motivo
# per cui i dati sono nulli in quelle domande. Pertanto,
# abbiamo scelto di includere una nuova categoria nelle domande (con il numero 6),
# per indicare i campi che erano nulli.

#STAMPA DISTIBUZIONE COLONNE

#Come possiamo vedere, la maggior parte delle domande ha una distribuzione quasi normale,
# con la maggior parte delle risposte concentrate tra 2, 3 e 4
# (D'accordo, non sono né d'accordo né in disaccordo e non sono d'accordo).
# Tuttavia, ci sono alcune domande su cui la maggior parte delle risposte si è concentrata in forte disaccordo,
# come: Use1, Use2, Vis3 e Exp5. Questi casi si riferiscono all'uso di wikipedia per l'elaborazione di materiale
# e attività educative, la citazione di wikipedia in lavori accademici,
# l'uso di wikipedia per lavorare con gli studenti, tra gli altri.

#Osservando l'ultima colonna dei grafici sopra, possiamo vedere la quantità di dati mancanti per ogni domanda.
# Si noti che PEU3, Vis1 e Vis2 hanno un numero maggiore di risposte mancanti,
# quando si analizzano rispettivamente queste domande,
# "È facile aggiungere o modificare informazioni su Wikipedia",
# "Wikipedia migliora la visibilità del lavoro degli studenti" e
# "È facile avere un registro dei contributi fatti su Wikipedia ".
# Puoi vedere che queste sono domande per gli utenti di wikipedia e
# la maggior parte degli insegnanti non è registrata sulla piattaforma, come visto in precedenza.
# Pertanto, è comprensibile che queste domande abbiano un maggior numero di domande senza risposta.

In [29]:
data = data.iloc[:, 9:].fillna(0)
data = data.astype(int)


In [30]:
# time su ogni op
#time su foreach
#time su for each per val

In [31]:
# funzione built-in
# %timeit data['Qu4'].map({0:0, 1:5, 2:4, 3:3, 4:2, 5:1})

In [32]:
# fancy indexing e combine
# %timeit main.reverse_vote_uf(data['Qu4'])

In [33]:
# foreach con switcher
# %timeit main.reverse_vote_not_uf(data['Qu4'].array)


In [34]:
data['Qu4'] = data['Qu4'].map({0:0, 1:5, 2:4, 3:3, 4:2, 5:1})

In [35]:
data

,USERWIKI,PU1,PU2,PU3,PEU1,PEU2,PEU3,ENJ1,ENJ2,Qu1,Qu2,Qu3,Qu4,Qu5,Vis1,Vis2,Vis3,Im1,Im2,Im3,SA1,SA2,SA3,Use1,Use2,Use3,Use4,Use5,Pf1,Pf2,Pf3,JR1,JR2,BI1,BI2,Inc1,Inc2,Inc3,Inc4,Exp1,Exp2,Exp3,Exp4,Exp5
0,0,4,4,3,5,5,3,4,4,3,3,2,4,3,3,3,1,2,4,2,4,4,4,3,1,1,2,4,1,1,3,4,1,2,3,5,5,5,5,4,4,4,1,2
1,0,2,3,3,4,4,3,3,4,4,4,3,3,2,3,3,1,1,2,1,4,4,4,2,2,1,1,2,1,1,1,4,3,2,2,4,4,3,4,2,2,4,2,4
2,0,2,2,2,4,4,3,3,3,2,2,2,1,3,2,3,2,2,4,3,4,2,4,1,1,1,1,1,3,3,3,4,3,1,1,5,3,5,5,2,2,2,1,3
3,0,3,3,4,3,3,3,4,3,3,4,3,3,3,3,4,3,2,2,3,5,5,5,3,3,3,3,3,4,4,4,5,4,3,3,3,4,4,3,4,4,3,3,4
4,1,4,3,5,5,4,3,4,4,4,5,4,3,4,4,4,4,3,2,3,5,5,5,3,3,5,5,5,5,5,5,3,3,5,5,5,5,4,4,5,5,5,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,0,3,3,3,5,5,2,4,5,3,3,4,1,2,2,2,1,2,2,2,3,3,4,0,0,2,2,2,1,2,1,5,4,2,2,2,2,2,2,0,0,0,0,0
909,0,3,3,4,5,4,3,4,4,4,4,4,3,3,2,3,3,2,0,2,0,4,5,3,2,3,3,5,2,2,4,3,3,4,4,4,3,3,4,4,4,4,1,1
910,0,3,3,3,5,4,3,3,4,3,3,2,1,2,2,4,1,1,2,0,4,5,4,3,1,1,1,3,5,5,3,3,1,2,2,5,4,3,0,5,5,5,4,1
911,0,3,3,5,5,4,2,4,4,4,4,3,4,5,3,2,5,4,5,4,5,5,5,3,1,5,5,0,1,2,4,4,2,5,5,1,5,2,2,4,4,2,1,1


In [36]:
#normalizza colone
